In [ ]:
import wikipedia


def getWikiPage(link):
    try:
        composer = wikipedia.page(title=link, auto_suggest=False)
        return composer
    except:
        pass
    try:
        composer = wikipedia.page(title=link, auto_suggest=True)
        return composer
    except:
        return None

links_set = set()
links_set |= set(getWikiPage("List_of_medieval_composers").links)
links_set |= set(getWikiPage("List_of_Renaissance_composers").links)
links_set |= set(getWikiPage("List_of_Baroque_composers").links)
links_set |= set(getWikiPage("List_of_Classical-era_composers").links)
links_set |= set(getWikiPage("List_of_Romantic-era_composers").links)
links_set |= set(getWikiPage("List_of_20th-century_classical_composers").links)
links_set |= set(getWikiPage("List_of_21st-century_classical_composers").links)
links = list(links_set)

blacklist = ("romantic music", "list of")
relations = {}
index = 0
for link in links:
    if link.lower().startswith(blacklist):
        continue
    
    composer = getWikiPage(link)
    if composer == None:
        continue
    index += 1
    if index % 50 == 0:
        print(index)
    for category in composer.categories:
        if 'composer' in category.lower():
            relations[link] = composer.links
            break
allComposers = relations.keys()


50
100
150
200
250
300
350
400
450


In [ ]:
def filterComposerRelationsOnly(relationsOfComposer, allComposers):
    return [relation for relation in relationsOfComposer if relation in allComposers]

relations = {k: filterComposerRelationsOnly(relationsOfComposer, allComposers) for k, relationsOfComposer in relations.items()}


In [ ]:
file = open("followerFolloweeRomantic.txt","w") 
for followeeComposer,followedComposers in relations.items():
    for followedComposer in followedComposers:
        file.write(followeeComposer+","+followedComposer+"\n")  
file.close() 